In [103]:
# Import dependencies
import requests
from pprint import pprint
import pandas as pd
from datetime import datetime as dt
import sqlalchemy

### Extract the Data 

Make an API call to the database to fetch the data for February 2023.

In [2]:
# Define the URL for the database
URL = "https://data.cityofnewyork.us/resource/h9gi-nx95.json?"

In [4]:
limit = "&$limit=10000"
query = "&$where=crash_date >= '2023-02-01T00:00:00.000' AND crash_date < '2023-03-01T00:00:00.000'"
query_url = URL+limit+query
response = requests.get(query_url)
data = response.json()

In [5]:
type(data)

list

In [6]:
len(data)

6821

In [14]:
pprint(data[8])

{'collision_id': '4602452',
 'contributing_factor_vehicle_1': 'Driver Inattention/Distraction',
 'contributing_factor_vehicle_2': 'Unspecified',
 'crash_date': '2023-02-01T00:00:00.000',
 'crash_time': '0:55',
 'latitude': '40.74478',
 'location': {'human_address': '{"address": "", "city": "", "state": "", '
                               '"zip": ""}',
              'latitude': '40.74478',
              'longitude': '-73.93262'},
 'longitude': '-73.93262',
 'number_of_cyclist_injured': '0',
 'number_of_cyclist_killed': '0',
 'number_of_motorist_injured': '0',
 'number_of_motorist_killed': '0',
 'number_of_pedestrians_injured': '0',
 'number_of_pedestrians_killed': '0',
 'number_of_persons_injured': '0',
 'number_of_persons_killed': '0',
 'off_street_name': '32 PLACE',
 'on_street_name': 'QUEENS BOULEVARD',
 'vehicle_type_code1': 'Sedan',
 'vehicle_type_code2': 'Pick-up Truck'}


In [22]:
df = pd.DataFrame(data)
df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01T00:00:00.000,12:30,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01T00:00:00.000,18:34,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01T00:00:00.000,11:00,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01T00:00:00.000,18:15,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01T00:00:00.000,9:44,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


## Transform

crash_date to date format, crash_time to timestamp, numbers to integers

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6821 entries, 0 to 6820
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6821 non-null   object
 1   crash_time                     6821 non-null   object
 2   borough                        4479 non-null   object
 3   zip_code                       4478 non-null   object
 4   latitude                       6354 non-null   object
 5   longitude                      6354 non-null   object
 6   location                       6354 non-null   object
 7   cross_street_name              1833 non-null   object
 8   number_of_persons_injured      6821 non-null   object
 9   number_of_persons_killed       6821 non-null   object
 10  number_of_pedestrians_injured  6821 non-null   object
 11  number_of_pedestrians_killed   6821 non-null   object
 12  number_of_cyclist_injured      6821 non-null   object
 13  num

In [78]:
df_dropped = df[df['vehicle_type_code1'].notna()]
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6718 entries, 0 to 6820
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     6718 non-null   object
 1   crash_time                     6718 non-null   object
 2   borough                        4396 non-null   object
 3   zip_code                       4395 non-null   object
 4   latitude                       6252 non-null   object
 5   longitude                      6252 non-null   object
 6   location                       6252 non-null   object
 7   cross_street_name              1813 non-null   object
 8   number_of_persons_injured      6718 non-null   object
 9   number_of_persons_killed       6718 non-null   object
 10  number_of_pedestrians_injured  6718 non-null   object
 11  number_of_pedestrians_killed   6718 non-null   object
 12  number_of_cyclist_injured      6718 non-null   object
 13  num

In [79]:
df_dropped['crash_date']= pd.to_datetime(df_dropped['crash_date'])
df_dropped.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,12:30,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,18:34,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,11:00,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,18:15,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01,9:44,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df_dropped['crash_time'] = pd.to_datetime(df_dropped['crash_time'], format='%H:%M').dt.hour
df_dropped.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code1,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2023-02-01,12,BROOKLYN,11215,40.666996,-73.9923,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Sedan,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,18,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Taxi,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,11,BROOKLYN,11215,40.67216,-73.98721,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,Garbage or Refuse,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-01,18,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-01,9,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Station Wagon/Sport Utility Vehicle,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
columns_to_int = ['number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
                  'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured', 'number_of_motorist_killed']
df_dropped[columns_to_int] = df_dropped[columns_to_int].astype(int)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6718 entries, 0 to 6820
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6718 non-null   datetime64[ns]
 1   crash_time                     6718 non-null   int64         
 2   borough                        4396 non-null   object        
 3   zip_code                       4395 non-null   object        
 4   latitude                       6252 non-null   object        
 5   longitude                      6252 non-null   object        
 6   location                       6252 non-null   object        
 7   cross_street_name              1813 non-null   object        
 8   number_of_persons_injured      6718 non-null   int64         
 9   number_of_persons_killed       6718 non-null   int64         
 10  number_of_pedestrians_injured  6718 non-null   int64         
 11  number_of_pedestr

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [82]:
df_dropped[['latitude','longitude']] = df_dropped[['latitude','longitude']].astype(float)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6718 entries, 0 to 6820
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     6718 non-null   datetime64[ns]
 1   crash_time                     6718 non-null   int64         
 2   borough                        4396 non-null   object        
 3   zip_code                       4395 non-null   object        
 4   latitude                       6252 non-null   float64       
 5   longitude                      6252 non-null   float64       
 6   location                       6252 non-null   object        
 7   cross_street_name              1813 non-null   object        
 8   number_of_persons_injured      6718 non-null   int64         
 9   number_of_persons_killed       6718 non-null   int64         
 10  number_of_pedestrians_injured  6718 non-null   int64         
 11  number_of_pedestr

In [101]:
df_binned = df_dropped
bins = [0,4,8,12,16,20,24]
labels = ["12AM - 4AM","4AM - 8AM","8AM - 12PM","12PM - 4PM","4PM - 8PM","8PM - 12AM"]
df_binned['period_of_day'] = pd.cut(x = df_binned['crash_time'], bins = bins, labels = labels)
df_binned.head()

/Users/yashkansal97/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,cross_street_name,number_of_persons_injured,number_of_persons_killed,...,vehicle_type_code2,on_street_name,off_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5,period_of_day
0,2023-02-01,12,BROOKLYN,11215,40.666996,-73.992300,"{'latitude': '40.666996', 'longitude': '-73.99...",136 15 STREET,0,0,...,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM
1,2023-02-01,18,NaN,NaN,40.754177,-73.984604,"{'latitude': '40.754177', 'longitude': '-73.98...",NaN,0,0,...,Sedan,WEST 41 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
2,2023-02-01,11,BROOKLYN,11215,40.672160,-73.987210,"{'latitude': '40.67216', 'longitude': '-73.987...",NaN,0,0,...,NaN,4 AVENUE,6 STREET,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM
3,2023-02-01,18,MANHATTAN,10022,40.758465,-73.970924,"{'latitude': '40.758465', 'longitude': '-73.97...",601 LEXINGTON AVENUE,1,0,...,Bike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4PM - 8PM
4,2023-02-01,9,BROOKLYN,11229,NaN,NaN,NaN,NaN,1,0,...,Box Truck,KINGS HIGHWAY,BEDFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,8AM - 12PM


In [104]:
engine=sqlalchemy.create_engine('sqlite:///data/db.sqlite')

In [106]:
df_binned.to_sql('data', if_exists='replace', index=False, con=engine)

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 6 - probably unsupported type.
[SQL: INSERT INTO data (crash_date, crash_time, borough, zip_code, latitude, longitude, location, cross_street_name, number_of_persons_injured, number_of_persons_killed, number_of_pedestrians_injured, number_of_pedestrians_killed, number_of_cyclist_injured, number_of_cyclist_killed, number_of_motorist_injured, number_of_motorist_killed, contributing_factor_vehicle_1, contributing_factor_vehicle_2, collision_id, vehicle_type_code1, vehicle_type_code2, on_street_name, off_street_name, contributing_factor_vehicle_3, vehicle_type_code_3, contributing_factor_vehicle_4, vehicle_type_code_4, contributing_factor_vehicle_5, vehicle_type_code_5, period_of_day) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('2023-02-01 00:00:00.000000', 12, 'BROOKLYN', '11215', 40.666996, -73.9923, {'latitude': '40.666996', 'longitude': '-73.9923', 'hum